In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [3]:
df=pd.read_excel('augmented_dataset_shuffled.xlsx')

In [4]:
# Définir les features (X) et la cible (y)
X = df[['cpu_usage','ram_usage','disk_usage', 'level', 'temperature','read_errors','write_errors','reallocated_sectors', 'event_id']] 
y = df['label']

In [5]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Encodage de la feature (level)
level_encoder = LabelEncoder()  # Nouvelle instance !
df['level'] = level_encoder.fit_transform(df['level'])  # 3 classes
print('bien')

bien


In [14]:
# Encodage des labels si ce sont des strings (ex: ["A", "B", "C"] → [0, 1, 2])
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)
print('bien')


bien


In [16]:
# 1. Entraînement d'un modèle KNN avec des hyperparamètres par défaut
knn_default = KNeighborsClassifier()
knn_default.fit(X_train, y_train)

ValueError: could not convert string to float: 'Error'

In [ ]:
# 2. Calcul de l'accuracy sur les ensembles d'entraînement et de test
y_train_pred = knn_default.predict(X_train)
y_test_pred = knn_default.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Accuracy sur l'entraînement (KNN par défaut) :", train_accuracy)
print("Accuracy sur le test (KNN par défaut) :", test_accuracy)


In [ ]:
# 3. Ajustement des hyperparamètres avec Random Search
param_dist = {
    'n_neighbors': [3, 5, 7, 10, 15],  # Nombre de voisins
    'weights': ['uniform', 'distance'],  # Poids des voisins
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithme de recherche
    'p': [1, 2]  # Distance (1 pour Manhattan, 2 pour Euclidienne)
}

random_search = RandomizedSearchCV(
    estimator=KNeighborsClassifier(),
    param_distributions=param_dist,
    n_iter=20,  # Nombre de combinaisons à tester
    cv=3,  # 5-fold cross-validation
    scoring='accuracy',
    n_jobs=-1,  # Utiliser tous les cœurs disponibles
    random_state=42
)

random_search.fit(X_train, y_train)

In [ ]:
# 4. Meilleurs hyperparamètres trouvés par Random Search
best_params_knn = random_search.best_params_
print("Meilleurs hyperparamètres (KNN Random Search) :", best_params_knn)

In [ ]:
# 5. Évaluation du meilleur modèle sur l'ensemble de test
best_model_knn = random_search.best_estimator_
y_test_pred_knn = best_model_knn.predict(X_test)
test_accuracy_knn = accuracy_score(y_test, y_test_pred_knn)
print("Accuracy sur le test (KNN optimisé) :", test_accuracy_knn)


In [ ]:
# 6. Matrice de confusion pour KNN
cm_knn = confusion_matrix(y_test, y_test_pred_knn)
print("\nMatrice de confusion (KNN) :")
print(cm_knn)

In [12]:
# 7. Visualisation de la matrice de confusion
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues', xticklabels=X, yticklabels=y)
plt.xlabel('Prédit')
plt.ylabel('Vrai')
plt.title('Matrice de confusion (KNN)')
plt.show()


ValueError: The number of FixedLocator locations (20000), usually from a call to set_ticks, does not match the number of labels (7).

Error in callback <function _draw_all_if_interactive at 0x000002910565E050> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000002910812B250> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

In [ ]:
# 8. Rapport de classification pour KNN
print("\nRapport de classification (KNN) :")
print(classification_report(y_test, y_test_pred_knn, target_names=data.target_names))